<div style='background-color: #6c3d75; align:center;color:white;border: 4px solid ; text-align:center;margin: 5px;padding:5px;font-size:20px'>Sales Price Prediction Challenge - Step 2 - Analyze data</div>
<div style="text-align: center;">
<img height="150" width="800"  src="https://sp-ao.shortpixel.ai/client/q_glossy,ret_img,w_1000/https://www.leadsquared.com/wp-content/uploads/2019/02/banner-4.png" alt="C1 Technologies"></div>

<h4 style="color:red;text-align:center"> * * * Please upvote if you like this kernel. * * *</h4>  
  <h4>This is step 2, Please visit <a href='https://www.kaggle.com/zenstat/notebook-for-beginners-step-1-load-the-data' > step 1 kernal code </a>to understand the basics of loading data. </h4><h4>In this kernal, we will work on second step of the anaysis which is to analyze and understand all variables. We will learn <br><br>
    * merge and join <br><br>
    * data quality checks - missing data, outliers <br><br>
    * data transformation - MVI - missing value imputations<br><br>
    We will load the data same as in <a href='https://www.kaggle.com/zenstat/notebook-for-beginners-step-1-load-the-data' >step 1 kernel</a> Be sure to check it out if you haven't.
</h4>

In [ ]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
#load items data 
items = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
#load items category data
itemscat = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
#load shops data
shops = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
#load sales train data  
train = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
#load sales test data  
test = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")

<h4>lets quickly take a look at individual datasets. 

In [ ]:
print("************************** items table **************************\n", items.head(1),"\n******************************************************************************\n")
print("************************** items category table **************************\n",itemscat.head(1),"\n******************************************************************************\n")
print("************************** shops table **************************\n",shops.head(1),"\n******************************************************************************\n")
print("************************** train table **************************\n",train.head(1),"\n******************************************************************************\n")
print("************************** test table **************************\n",test.head(1),"\n******************************************************************************\n")

<h4>Before we begin data analysis, we first need a complete dataset that contains all variables we have for prediction. 
 <h4>Here the datasets we are given are scattered and needs to be merged. So lets merge all datasets one by one and create a final dataset by merging all variables.

In [ ]:
train_itms= train.join(items, on='item_id', rsuffix='_')

In [ ]:
train_itms_shps = train_itms.join(shops, on='shop_id', rsuffix='_')

In [ ]:
train_itms_shps_cat = train_itms_shps.join(itemscat, on='item_category_id', rsuffix='_')

In [ ]:
#drop duplicate columns 
train_itms_shps_cat = train_itms_shps.join(itemscat, on='item_category_id', rsuffix='_')

In [ ]:
train_data = train_itms_shps_cat.drop(["item_id_","shop_id_","item_category_id_" ], axis=1)

<h3>Here is a cool thing about python. You can do all above merge steps in one line

In [ ]:
train_data=train.join(items, on='item_id', rsuffix='_').join(shops, on='shop_id', rsuffix='_').join(itemscat, on='item_category_id', rsuffix='_').drop(["item_id_","shop_id_","item_category_id_" ], axis=1)

<h3> Also, let's change data type for date field same way as first kernel.

In [ ]:
# lets convert date data type to date field.
train_data["date"]=pd.to_datetime(train_data["date"])
#create year and year-month field so we can summarise the data by dates
train_data["date_yyyy"]=pd.to_datetime(train_data["date"]).dt.strftime('%Y') 
train_data["date_yyyymm"]=pd.to_datetime(train_data["date"]).dt.strftime('%Y-%m') 

<h3>How to check if there any duplicates in the datasets? 

In [ ]:
#all duplcates will be stored in duplicate dataframe
duplicate = train_data[train_data.duplicated()]

<h3>Check duplicate dataset and you will find 6 records that are duplicate. Below is how we can find and then get rid of duplicates

In [ ]:
#we will get rid of the duplicates using 
train_final= train_data.drop_duplicates()

In [ ]:
# train_final is the data without any duplicates
train_final.shape

<h3>Are there any outliers in the dataset

In [ ]:
import seaborn as sns
sns.boxplot(x=train_final['item_price'])

<h3>if you look carefully at item price column, there is a single item price which is $307,980. while mostly item price are less than 50k. there are only 3 items greater than 50k.
<br><br>Same way, the count seem to have some exceptionally high values. this time we will treat them both. 


In [ ]:
#train_final[train_final["item_count"]>40000].head(30).reset_index() 
train_final[train_final["item_cnt_day"]>40000].head(30).reset_index() 

In [ ]:
sns.boxplot(x=train_final['item_cnt_day'])

In [ ]:
train_final[train_final["item_cnt_day"]>500].head(30).reset_index()

### Here we will treat the outliers and will cap them around 99th percentile. 

<h3><span style="color:red"> Please be warned before doing this. </span> ALWAYS ALWAYS study outliers, they give insight in to process, business and outliers provide good information that will help in building statistical models aka ML algos.

In [ ]:
#WIP, 
train_final['item_cnt_day'] = np.where(train_final['item_cnt_day']>400 , 400,train_final['item_cnt_day'] )

#cap item count >500 to 500 
#cap item price >40000 to 40000 
train_final['item_price'] = np.where(train_final['item_price']>40000 , 40000,train_final['item_price'] )

#another way to do same in python, try . there are tons of other ways!!! 
#train_final['item_cnt_day'] = [400 if x > 400 else x for x in train_final['item_cnt_day']]

In [ ]:
sns.boxplot(x=train_final['item_cnt_day'])

In [ ]:
sns.boxplot(x=train_final['item_price'])

<h5>How to check if there any missing value in any of the variable? 

In [ ]:
train_data.isnull().sum()

<h5>All above variables are showing zero's that means there are no variables with missing values. This is a good thing, <span style="color:red"> We are in luck, we can skip missing value treatment step !!!</h5><h5>lets summarize all sales by date variable to see if we can learn something. 

In [ ]:
train_year = train_data.groupby(['date_yyyy'])["item_price","item_cnt_day"].sum().reset_index().sort_values('date_yyyy', ascending=True)
train_year.T

In [ ]:
train_month = train_data.groupby(['date_yyyymm'])["item_price","item_cnt_day"].sum().reset_index().sort_values('date_yyyymm', ascending=True)
train_month.T

<h5>but this doesnt look very intuitive, a visual is the easiest way to share the story. <br><br>
Lets display the same details on bar plot and see if it looks better. First lets look at yearly trend of total sales

In [ ]:
import seaborn as sns  #import seaborn
sns.set(rc={'figure.figsize':(5.7,5.27)})
import matplotlib.pylab as plt 
plt.xticks(rotation=45)
sns.barplot(x='date_yyyy', y='item_price', data=train_year)

<h5>We can clearly see 2014 has been the best year in terms of sales. Lets deep dive further and look in to monthly trend

In [ ]:
#Historic sales items per day
sns.set(rc={'figure.figsize':(20,6)})
import matplotlib.pylab as plt
plt.xticks(rotation=45)
sns.barplot(x='date_yyyymm', y='item_price', data=train_month)

<h5>As you can see, there are two peaks in the data and they are present in last two months of each year. <BR><BR>This is because of festival season and people are buying a lot in the last couple of months. this is also attributed to a lot of discounts that are provided during festival season. 
<H5> does sales item show the same trend. Lets see 

In [ ]:
#Historic sales items count per day
sns.set(rc={'figure.figsize':(20,6)})
import matplotlib.pylab as plt
plt.xticks(rotation=45)
sns.barplot(x='date_yyyymm', y='item_cnt_day', data=train_month)

<h3 style="color:red;text-align:center"> * * *  to be continued * * * </h3>